<a href="https://colab.research.google.com/github/amitgupta226571/DEEP-LEARNING-/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Experiment 2 : Neural Network from Scratch using Numpy

In [1]:
from sklearn.datasets import fetch_openml
import numpy as np


In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.to_numpy()
y = mnist.target.astype(int).to_numpy()


In [3]:
X = X / 255.0


In [4]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]


In [5]:
def one_hot(y, num_classes=10):
    return np.eye(num_classes)[y]

y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)


In [6]:
np.random.seed(42)

W1 = np.random.randn(784, 128) * 0.01
b1 = np.zeros((1, 128))

W2 = np.random.randn(128, 10) * 0.01
b2 = np.zeros((1, 10))


In [7]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return (Z > 0).astype(float)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)


In [8]:
def forward(X):
    Z1 = X @ W1 + b1
    A1 = relu(Z1)

    Z2 = A1 @ W2 + b2
    A2 = softmax(Z2)

    return Z1, A1, A2


In [9]:
def cross_entropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))


In [10]:
def backward(X, y_true, Z1, A1, A2, lr=0.1):
    global W1, b1, W2, b2
    m = X.shape[0]

    dZ2 = A2 - y_true
    dW2 = A1.T @ dZ2 / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * relu_derivative(Z1)

    dW1 = X.T @ dZ1 / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2


In [11]:
epochs = 10
batch_size = 128
lr = 0.1

for epoch in range(epochs):
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train_oh[i:i+batch_size]

        Z1, A1, A2 = forward(X_batch)
        backward(X_batch, y_batch, Z1, A1, A2, lr)

    Z1, A1, A2 = forward(X_train[:1000])
    loss = cross_entropy(y_train_oh[:1000], A2)
    print(f"Epoch {epoch+1} | Loss: {loss:.4f}")


Epoch 1 | Loss: 0.3775
Epoch 2 | Loss: 0.2990
Epoch 3 | Loss: 0.2489
Epoch 4 | Loss: 0.2123
Epoch 5 | Loss: 0.1858
Epoch 6 | Loss: 0.1651
Epoch 7 | Loss: 0.1495
Epoch 8 | Loss: 0.1363
Epoch 9 | Loss: 0.1252
Epoch 10 | Loss: 0.1158


In [12]:
def accuracy(X, y):
    _, _, A2 = forward(X)
    preds = np.argmax(A2, axis=1)
    return np.mean(preds == y) * 100


In [13]:
print("Test Accuracy:", accuracy(X_test, y_test), "%")


Test Accuracy: 96.52 %


In [14]:
idx = 5
image = X_test[idx].reshape(1, -1)
_, _, probs = forward(image)

print("Predicted digit:", np.argmax(probs))
print("Actual digit:", y_test[idx])


Predicted digit: 1
Actual digit: 1
